# DEM & Hillshade Automation – Python Launcher for R Pipeline

This notebook automates the generation of a Digital Elevation Model (DEM) and hillshades for any area of interest, using a parameterized R script.  
You define your area (GeoJSON), set output directories, and control all processing options directly from Python.

---

## 🗺️ What does this pipeline do?
- Reads a polygon (GeoJSON) as area of interest
- Downloads and mosaics DEM tiles for the area (using the [elevatr](https://cran.r-project.org/web/packages/elevatr/index.html) package in R)
- Generates several hillshade layers in parallel (with configurable sun azimuths/altitudes)
- Optionally adds a normalized slope layer
- Blends all relief layers using custom weights
- Exports all outputs as GeoTIFFs in a folder you define
- Fully parameterized: all R script arguments are passed from Python, so you can batch/automate/run in reproducible pipelines

---

## 🛠️ Required Python packages
- No special dependencies (uses Python’s built-in `subprocess`)
- [Optional] For file checks: `os`

---

## 📝 Arguments passed to the R script

| Argument name        | Example value            | Description                                                             |
|----------------------|-------------------------|-------------------------------------------------------------------------|
| `--data_dir`         | `data_input/dem`        | Directory where all DEM and hillshade outputs will be written           |
| `--area_file`        | `interlaken_bbox.geojson`| Name (or path) of the GeoJSON file defining the area of interest        |
| `--dem_margin`       | `0.0045`                | Margin (in decimal degrees) added around the bounding box of the area   |
| `--dem_zoom`         | `12`                    | DEM zoom level (higher = higher resolution, but slower/larger download) |
| `--dem_export_name`  | `dem_elevatr.tif`       | Output filename for the DEM raster (inside `data_dir`)                  |
| `--blend_export_name`| `hillshades_blend.tif`  | Output filename for the blended hillshade (inside `data_dir`)           |
| `--hs_azimuths`      | `350,15,270`            | List of sun azimuths (directions), comma-separated, for hillshading     |
| `--hs_altitudes`     | `70,60,55`              | List of sun altitudes (elevations), comma-separated, for hillshading    |
| `--add_slope`        | `TRUE` or `FALSE`       | Whether to add a normalized slope layer to the stack                    |
| `--n_workers`        | `2`                     | Number of CPU cores to use for parallel processing                      |
| `--blend_weights`    | `0.6,0,0,0.6`           | Weights (comma-separated) for blending the hillshade and slope layers   |



### Inspiration

**This script’s multi-hillshade logic and parameter design are inspired by the QGIS “Batch Hillshader” plugin** (see screenshot).  
Parameters like azimuth, altitude, and even transparency follow the same logic for easy cross-software reproducibility.


# Example with a small portion of Interlaken (Switzerland) to test parameters on a small scale

In [11]:
import subprocess

# Définis tes paramètres ici (tous sont explicitement transmis) :
args = {
    "--data_dir":         "data_input/dem",
    "--area_file":        "interlaken_bbox.geojson",
    "--dem_margin":       0.0045,
    "--dem_zoom":         12,
    "--dem_export_name":  "dem_elevatr.tif",
    "--blend_export_name":"hillshades_blend.tif",
    "--hs_azimuths":      "350,15,270",
    "--hs_altitudes":     "70,60,55",
    "--add_slope":        "TRUE",           # ou FALSE
    "--n_workers":        2,
    "--blend_weights":    "0.6,0,0,0.6"
}

# Construction de la commande
r_script = "get_dem_hillshade_production.R"
cmd = ["Rscript", r_script]
for k, v in args.items():
    if v != "":
        cmd.extend([k, str(v)])

print(" ".join(cmd))  # Affiche la commande si tu veux debugger

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)


Rscript get_dem_hillshade_production.R --data_dir data_input/dem --area_file interlaken_bbox.geojson --dem_margin 0.0045 --dem_zoom 12 --dem_export_name dem_elevatr.tif --blend_export_name hillshades_blend.tif --hs_azimuths 350,15,270 --hs_altitudes 70,60,55 --add_slope TRUE --n_workers 2 --blend_weights 0.6,0,0,0.6
======== PARAMETRES UTILISES ========
Dossier travail      : data_input/dem
Fichier zone         : interlaken_bbox.geojson
Marge DEM            : 0.0045°
Zoom DEM             : 12
DEM export           : data_input/dem/dem_elevatr.tif
Blend export         : data_input/dem/hillshades_blend.tif
Azimuths hillshade   : 350,15,270
Altitudes hillshade  : 70,60,55
Add slope            : TRUE
Workers (CPU)        : 2
Blend weights        : 0.6,0,0,0.6

Reading layer `interlaken_bbox' from data source 
  `/Users/bajj@mediait.ch/Documents/LEDE_Program/Projets/2025_seisme_python/interlaken_bbox.geojson' 
  using driver `GeoJSON'
Simple feature collection with 1 feature and 0 fields
Geo

## Large scale example with Myanmar (it takes a lot of time to run)

In [ ]:
import subprocess

# Définis tes paramètres ici (tous sont explicitement transmis) :
args = {
    "--data_dir":         "data_input/dem",
    "--area_file":        "myanmar.json",
    "--dem_margin":       1,
    "--dem_zoom":         8,
    "--dem_export_name":  "dem_elevatr.tif",
    "--blend_export_name":"hillshades_blend.tif",
    "--hs_azimuths":      "350,15,270",
    "--hs_altitudes":     "70,60,55",
    "--add_slope":        "TRUE",           # ou FALSE
    "--aggregate_factor": "",               # vide = natif
    "--n_workers":        2,
    "--blend_weights":    "0.6,0,0,0.6"
}

# Construction de la commande
r_script = "get_dem_hillshade_production.R"
cmd = ["Rscript", r_script]
for k, v in args.items():
    if v != "":
        cmd.extend([k, str(v)])

print(" ".join(cmd))  # Affiche la commande si tu veux debugger

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)
